In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)  
        self.fc2 = nn.Linear(hidden_size, int(hidden_size / 2))
        self.dropout2 = nn.Dropout(dropout_rate)  
        self.fc3 = nn.Linear(int(hidden_size / 2), output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        # out = self.dropout1(out)  
        out = self.fc2(out)
        out = self.relu(out)
        # out = self.dropout2(out)  
        out = self.fc3(out)
        return out
    
input_size = 365*19
hidden_size = 1024
output_size = 1  
model = MLP(input_size, hidden_size, output_size)
model.to('cuda')
train_dataset = torch.load('../../../data/cleaned/train.pt')
train_x, train_y = train_dataset.tensors  

val_dataset = torch.load('../../../data/cleaned/val.pt')
test_x, test_y = val_dataset.tensors  

train_x = train_x.reshape(-1, 365*19)
test_x = test_x.reshape(-1, 365*19)

train_x = train_x.to('cuda')
test_x = test_x.to('cuda')
train_y = train_y.to('cuda')
test_y = test_y.to('cuda')

print(train_x.shape)
criterion = nn.MSELoss()  

optimizer = optim.Adam(model.parameters(), lr=5e-5)
num_epochs = 1500

for epoch in range(num_epochs):
    model.train()  
    outputs = model(train_x)
    loss = criterion(outputs, train_y.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        model.eval()  
        with torch.no_grad():
            val_outputs = model(test_x)
            val_loss = criterion(val_outputs, test_y.unsqueeze(1))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.sqrt(loss.item()):.4f}, Validation Loss: {np.sqrt(val_loss.item()):.4f}')

torch.Size([17724, 6935])
Epoch [100/1500], Loss: 287.4871, Validation Loss: 128.5077
Epoch [200/1500], Loss: 283.8199, Validation Loss: 128.4548
Epoch [300/1500], Loss: 276.9374, Validation Loss: 131.3308
Epoch [400/1500], Loss: 261.2257, Validation Loss: 130.9897
Epoch [500/1500], Loss: 228.1291, Validation Loss: 133.7019
Epoch [600/1500], Loss: 174.8842, Validation Loss: 133.7079
Epoch [700/1500], Loss: 123.7724, Validation Loss: 130.1123
Epoch [800/1500], Loss: 91.4500, Validation Loss: 123.0044
Epoch [900/1500], Loss: 72.5286, Validation Loss: 119.9525
Epoch [1000/1500], Loss: 62.1644, Validation Loss: 118.2955
Epoch [1100/1500], Loss: 55.7313, Validation Loss: 116.1381
Epoch [1200/1500], Loss: 51.1945, Validation Loss: 115.1271
Epoch [1300/1500], Loss: 47.4918, Validation Loss: 114.3789
Epoch [1400/1500], Loss: 44.1495, Validation Loss: 114.8366
Epoch [1500/1500], Loss: 40.9413, Validation Loss: 115.1793
